In [ ]:
%%capture

!pip install llama-index-llms-ibm==0.1.0 --user
!pip install llama-index-embeddings-ibm==0.1.0 --user
!pip install llama-index==0.10.65 --user

In [ ]:
# Use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from llama_index.llms.ibm import WatsonxLLM
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.ibm import WatsonxEmbeddings
from llama_index.core import VectorStoreIndex

In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/pfSOEORnYBZppsnhmZ1a8A/lora-paper.pdf"

In [ ]:
documents = SimpleDirectoryReader(input_files=["lora-paper.pdf"]).load_data()

In [ ]:
documents[0]

In [ ]:
splitter = SentenceSplitter(chunk_size=500)

In [ ]:
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
len(nodes)

In [ ]:
node_metadata = nodes[0].get_content(metadata_mode=True)
print(str(node_metadata))

In [ ]:
watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
    truncate_input_tokens=3,
)

In [ ]:
index = VectorStoreIndex(
    nodes=nodes, 
    embed_model=watsonx_embedding, 
    show_progress=True
)

In [ ]:
base_retriever = index.as_retriever(similarity_top_k=3) # 3 for top 3 results

source_nodes = base_retriever.retrieve("GPT-2") # querying about GPT-2

In [ ]:
for node in source_nodes:
    # print(node.metadata)
    print(f"---------------------------------------------")
    print(f"Score: {node.score:.3f}")
    print(node.get_content())
    print(f"---------------------------------------------\n\n")

In [ ]:
temperature = 0.1
max_new_tokens = 75
additional_params = {
    "decoding_method": "sample",
    "min_new_tokens": 1,
    "top_k": 50,
    "top_p": 1,
}

watsonx_llm = WatsonxLLM(
    model_id="ibm/granite-3-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
    temperature=temperature,
    max_new_tokens=max_new_tokens,
    additional_params=additional_params,
)